In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Load the cleaned complaint dataset
df = pd.read_csv('/Users/carinayan/Downloads/dcwp_complaints_project/data/cleaned_consumer_complaints.csv')

# Count the occurrences of each complaint resolution result
result_counts = df['Result'].value_counts()

# Keep only the top 6 most common results; group the rest into "Other"
top_n = 6
top_results = result_counts.iloc[:top_n]         # Top 6 categories
other_count = result_counts.iloc[top_n:].sum()   # Combine remaining categories into "Other"

# Combine the top results with the "Other" group into one DataFrame
summary_df = pd.concat([top_results, pd.Series({'Other': other_count})]).reset_index()
summary_df.columns = ['Result', 'Count']  # Rename columns

# Create a donut pie chart using Plotly Express
fig = px.pie(
    summary_df,
    names='Result',         # Category labels for each slice
    values='Count',         # Values for each slice
    title='Distribution of Complaint Resolution Results (Top 6 + Other)',  # Chart title
    hole=0.4                # Creates a donut-style chart with a center hole
)

# Enhance label appearance: show percentage and category inside each slice
fig.update_traces(textposition='inside', textinfo='percent+label')

# Display the chart
fig.show()


In [ ]:
df = pd.read_csv('/Users/carinayan/Downloads/dcwp_complaints_project/data/cleaned_consumer_complaints.csv')

# Group by Business Category and Result, and count the number of complaints in each group
grouped = df.groupby(['Business Category', 'Result']).size().reset_index(name='Count')

# Select the top 20 industries with the highest total complaint counts (optional filtering step)
top_industries = (
    grouped.groupby('Business Category')['Count']
    .sum()
    .sort_values(ascending=False)
    .head(20)
    .index
)
# Keep only rows where the Business Category is in the top 20
grouped = grouped[grouped['Business Category'].isin(top_industries)]

# Create a stacked bar chart with Business Category on the x-axis, complaint count on y-axis,
# and colors representing different resolution results
fig = px.bar(
    grouped,
    x='Business Category',
    y='Count',
    color='Result',
    title='Number of Complaints by Industry (Stacked by Result)',
    labels={'Count': 'Number of Complaints', 'Business Category': 'Industry'},
    barmode='stack',  # Stacked bar chart mode
    height=600
)

fig.update_layout(
    xaxis_tickangle=-45,
    xaxis_title='Industry',
    yaxis_title='Number of Complaints',
    legend_title='Result',
    margin=dict(l=50, r=50, t=80, b=150)
)
fig.show()


In [ ]:
df = pd.read_csv('/Users/carinayan/Downloads/dcwp_complaints_project/data/cleaned_consumer_complaints.csv')

# Group by Business Category and Complaint Code, then count the number of complaints in each group
heatmap_data = df.groupby(['Business Category', 'Complaint Code']).size().reset_index(name='Count')

# Keep only the top 20 most frequent complaint codes across all industries
top_complaints = (
    heatmap_data.groupby('Complaint Code')['Count']
    .sum()
    .sort_values(ascending=False)
    .head(20)
    .index
)
# Filter the dataset to only include rows with the top 20 complaint codes
heatmap_data = heatmap_data[heatmap_data['Complaint Code'].isin(top_complaints)]

# Sort industry labels (Y-axis) by total number of complaints
industry_order = (
    heatmap_data.groupby('Business Category')['Count']
    .sum()
    .sort_values(ascending=False)
    .index
)

# Sort complaint code labels (X-axis) by total number of complaints
complaint_order = (
    heatmap_data.groupby('Complaint Code')['Count']
    .sum()
    .sort_values(ascending=False)
    .index
)

pivot_table = heatmap_data.pivot_table(
    index='Business Category',
    columns='Complaint Code',
    values='Count',
    fill_value=0  # Fill missing values with 0
)

pivot_table = pivot_table.loc[industry_order, complaint_order]

fig = px.imshow(
    pivot_table,
    labels=dict(x="Complaint Type", y="Industry", color="Number of Complaints"),
    title="Heatmap of Complaint Types Across Industries (Sorted)",
    color_continuous_scale='Reds', 
    height=800
)

fig.update_layout(
    xaxis_side="bottom",
    margin=dict(l=50, r=50, t=80, b=120)
)

fig.show()


In [ ]:
df = pd.read_csv('/Users/carinayan/Downloads/dcwp_complaints_project/data/cleaned_consumer_complaints.csv')

# Convert "Intake Date" to datetime format and extract the month (as 'YYYY-MM' string)
df['Intake Date'] = pd.to_datetime(df['Intake Date'])
df['Month'] = df['Intake Date'].dt.to_period('M').astype(str)

# Identify the top 5 industries with the highest total complaint counts
top_industries = (
    df.groupby('Business Category')
    .size()
    .sort_values(ascending=False)
    .head(5)
    .index
)

# Keep only the data for these top 5 industries
df_top = df[df['Business Category'].isin(top_industries)]

# Group by month and business category to count complaints per month per industry
trend_data = (
    df_top.groupby(['Month', 'Business Category'])
    .size()
    .reset_index(name='Complaint Count')
)

# Create a multi-line chart showing complaint trends over time by industry
fig = px.line(
    trend_data,
    x='Month',
    y='Complaint Count',
    color='Business Category',
    title='Monthly Complaint Trends by Top 5 Industries',
    labels={'Month': 'Month', 'Complaint Count': 'Number of Complaints', 'Business Category': 'Industry'},
    markers=True,  
)

# Add Play/Pause buttons to control animation
fig.update_layout(
    updatemenus=[dict(
        type="buttons",
        buttons=[
            dict(
                label="Play",
                method="animate",
                args=[None, {
                    "frame": {"duration": 500, "redraw": True},
                    "fromcurrent": True
                }]
            ),
            dict(
                label="Pause",
                method="animate",
                args=[[None], {
                    "frame": {"duration": 0, "redraw": False},
                    "mode": "immediate",
                    "transition": {"duration": 0}
                }]
            )
        ]
    )],
    xaxis=dict(
        rangeslider=dict(visible=True),  # Add a slider for the x-axis (month)
        type="category"  
    ),
    height=600
)

fig.frames = [go.Frame(
    data=[
        go.Scatter(
            x=trend_data[trend_data['Month'] <= m]['Month'],
            y=trend_data[trend_data['Month'] <= m][trend_data['Business Category'] == bc]['Complaint Count'],
            mode='lines+markers',
            name=bc
        ) for bc in top_industries
    ],
    name=m
) for m in sorted(trend_data['Month'].unique())]


fig.show()


/var/folders/1w/m6bndrfn3h96xky2vx7hh40r0000gn/T/ipykernel_47556/2539225705.py:68: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/var/folders/1w/m6bndrfn3h96xky2vx7hh40r0000gn/T/ipykernel_47556/2539225705.py:68: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/var/folders/1w/m6bndrfn3h96xky2vx7hh40r0000gn/T/ipykernel_47556/2539225705.py:68: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/var/folders/1w/m6bndrfn3h96xky2vx7hh40r0000gn/T/ipykernel_47556/2539225705.py:68: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/var/folders/1w/m6bndrfn3h96xky2vx7hh40r0000gn/T/ipykernel_47556/2539225705.py:68: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/var/folders/1w/m6bndrfn3h96xky2vx7hh40r0000gn/T/ipykernel_47556/2539225705.py:68: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/var/folders/1w/m6bndrfn3h96xky2vx

In [ ]:
df = pd.read_csv('/Users/carinayan/Downloads/dcwp_complaints_project/data/cleaned_consumer_complaints.csv')

df['Postcode'] = df['Postcode'].astype(str)

# Keep only ZIP codes in the New York City area (starting with '1' and 5 digits total)
df = df[df['Postcode'].str.match(r'^1\d{4}$')]

# Fill missing borough values with "Unknown" to avoid null issues in visualization
df['Borough'] = df['Borough'].fillna('Unknown')

# Group the data by ZIP code and Borough to count the number of complaints in each area
zipcode_counts = (
    df.groupby(['Postcode', 'Borough'])
    .size()
    .reset_index(name='Complaint Count')
)

# Create a choropleth map using Plotly Express and a public GeoJSON ZIP code file
fig = px.choropleth_mapbox(
    zipcode_counts,
    geojson="https://raw.githubusercontent.com/OpenDataDE/State-zip-code-GeoJSON/master/ny_new_york_zip_codes_geo.min.json",  # GeoJSON boundary file for NYC ZIP codes
    locations='Postcode',                   # Column in the data that matches the ZIP code
    featureidkey="properties.ZCTA5CE10",   
    color='Complaint Count',                # Color intensity based on complaint count
    color_continuous_scale="YlOrRd",    
    mapbox_style="carto-positron",       
    zoom=9,                                 # Initial zoom level on NYC
    center={"lat": 40.7128, "lon": -74.0060},  # Center the map on NYC coordinates
    opacity=0.6,                            # Set layer transparency
    labels={'Complaint Count': 'Number of Complaints'},  
    hover_data={'Postcode': True, 'Borough': True, 'Complaint Count': True},  # Info on hover
    title='Complaint Distribution by ZIP Code and Borough in NYC'
)

fig.update_layout(margin={"r":0,"t":50,"l":0,"b":0})
fig.show()


/var/folders/1w/m6bndrfn3h96xky2vx7hh40r0000gn/T/ipykernel_47556/1837261538.py:25: DeprecationWarning:

*choropleth_mapbox* is deprecated! Use *choropleth_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



In [ ]:
df = pd.read_csv('/Users/carinayan/Downloads/dcwp_complaints_project/data/cleaned_consumer_complaints.csv')

# Ensure numeric fields are properly converted to numeric values and fill any invalid/missing values with 0
df['Refund Amount'] = pd.to_numeric(df['Refund Amount'], errors='coerce').fillna(0)
df['Contract Cancelled Amount'] = pd.to_numeric(df['Contract Cancelled Amount'], errors='coerce').fillna(0)

# Compute total compensation by summing refund and contract cancellation amounts
df['Total Compensation'] = df['Refund Amount'] + df['Contract Cancelled Amount']

# Group by complaint type and calculate:
# - total number of complaints
# - average compensation
# - average satisfaction rate (as a proportion)
summary = (
    df.groupby('Complaint Code')
    .agg(
        Complaint_Count=('Complaint Code', 'count'),
        Average_Compensation=('Total Compensation', 'mean'),
        Satisfaction_Rate=('Satisfaction', 'mean')
    )
    .reset_index()
)

# Convert satisfaction rate to percentage and round to 2 decimal places
summary['Satisfaction_Rate'] = (summary['Satisfaction_Rate'] * 100).round(2)

# Keep only the top 10 complaint types by count
summary = summary.sort_values('Complaint_Count', ascending=False).head(10)

# Round average compensation values to 2 decimal places for display
summary['Average_Compensation'] = summary['Average_Compensation'].round(2)

fig = go.Figure(data=[go.Table(
    header=dict(
        values=["Complaint Type", "Complaint Count", "Average Compensation ($)", "Satisfaction Rate (%)"],
        fill_color='paleturquoise',  
        align='left'
    ),
    cells=dict(
        values=[
            summary['Complaint Code'],
            summary['Complaint_Count'],
            summary['Average_Compensation'],
            summary['Satisfaction_Rate']
        ],
        fill_color='lavender',  
        align='left'
    )
)])

fig.update_layout(
    title="Summary Table: Top 10 Complaint Types",
    margin=dict(l=20, r=20, t=80, b=20)
)


fig.show()


In [ ]:
df = pd.read_csv('/Users/carinayan/Downloads/dcwp_complaints_project/data/cleaned_consumer_complaints.csv')

# Convert 'Intake Date' and 'Result Date' columns to datetime format
df['Intake Date'] = pd.to_datetime(df['Intake Date'], errors='coerce')
df['Result Date'] = pd.to_datetime(df['Result Date'], errors='coerce')

# Calculate the number of days taken to resolve each complaint
df['Processing Days'] = (df['Result Date'] - df['Intake Date']).dt.days

# Filter out rows with invalid or extreme values (negative or over 500 days)
df = df[(df['Processing Days'] >= 0) & (df['Processing Days'] <= 500)]

fig = px.histogram(
    df,
    x='Processing Days',
    nbins=50,  
    title='Distribution of Complaint Processing Time (Days)',
    labels={'Processing Days': 'Days to Resolve Complaint'}
)

fig.update_layout(bargap=0.1)

fig.show()


In [ ]:
df = pd.read_csv('/Users/carinayan/Downloads/dcwp_complaints_project/data/cleaned_consumer_complaints.csv')

df['Intake Date'] = pd.to_datetime(df['Intake Date'], errors='coerce')
df['Result Date'] = pd.to_datetime(df['Result Date'], errors='coerce')

# Calculate the number of days it took to process each complaint
df['Processing Days'] = (df['Result Date'] - df['Intake Date']).dt.days

# Remove rows with invalid or extreme processing times (e.g., negative or over 500 days)
df = df[(df['Processing Days'] >= 0) & (df['Processing Days'] <= 500)]

fig = px.histogram(
    df,
    x='Processing Days',
    nbins=50,  
    title='Distribution of Complaint Processing Time (Days)',
    labels={'Processing Days': 'Days to Resolve Complaint'}
)

fig.update_layout(bargap=0.1)
fig.show()
